# 载入数据

In [1]:
import tensorflow as tf
#导入Tensorflow提供的读取MNIST的模块
import tensorflow.examples.tutorials.mnist.input_data as input_data
#读取MNIST数据
mnist = input_data.read_data_sets("MNIST_data", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


# 建立模型

In [2]:
#定义全连接层函数
def fcn_layer(inputs,           #输入数据
              input_dim,        #输入神经元数量
              output_dim,       #输出神经元数量
              activation=None): #激活函数
    W = tf.Variable(tf.truncated_normal([input_dim, output_dim], stddev=0.1)) #以截断正态分布的随机数初始化W
    b = tf.Variable(tf.zeros([output_dim])) #以0初始化b
    XWb = tf.matmul(inputs, W) + b #建立表达式: inputs * W + b
    if activation is None: #默认有使用激活函数
        outputs = XWb
    else: # 若传入激活函数，则用其对输出结果进行变换
        outputs = activation(XWb)
    return outputs

# 构建输入层

In [3]:
x = tf.placeholder(tf.float32, [None, 784], name="X")

# 构建模型

In [4]:
H1_NN = 256 # 第1隐藏层神经元为256
H2_NN = 64  # 第2隐藏层神经元为64

### 构建隐藏层1

In [5]:
h1 = fcn_layer(inputs=x,
               input_dim=784,
               output_dim=H1_NN,
               activation=tf.nn.relu)

### 构建隐藏层2

In [6]:
h2 = fcn_layer(inputs=h1,
               input_dim=H1_NN,
               output_dim=H2_NN,
               activation=tf.nn.relu)

### 构建输出层

In [7]:
forward = fcn_layer(inputs=h2,
               input_dim=H2_NN,
               output_dim=10,
               activation=None)
pred = tf.nn.softmax(forward)

# 训练模型

### 定义标签数据占位符

In [8]:
y = tf.placeholder(tf.float32, [None, 10], name="Y")

### 定义损失函数

In [9]:
#交叉熵
loss_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=forward, labels=y))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



### 设置训练参数

In [10]:
train_epochs = 40
batch_size = 50
total_batch = int(mnist.train.num_examples/batch_size)
display_step = 1
learning_rate = 0.01

### 选择优化器

In [11]:
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss_function)

### 定义准确率

In [12]:
correct_prediction = tf.equal(tf.argmax(y, 1),tf.argmax(pred, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

### 训练模型

In [13]:
#记录训练开始时间
from time import time
startTime = time()
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for epoch in range(train_epochs):
    for batch in range(total_batch):
        xs, ys = mnist.train.next_batch(batch_size)
        sess.run(optimizer,feed_dict={x: xs, y: ys})
    #total_batch个批次训练完成后，使用验证数据计算误差与准确率
    loss, acc = sess.run([loss_function, accuracy],feed_dict={x: mnist.validation.images,
                                                              y: mnist.validation.labels})
    if(epoch+1) % display_step ==0:
        print("Train Epoch:","%02d" % (epoch+1),
              "Loss=","{:.9f}".format(loss), " Accuracy=","{:.4f}".format(acc))
#运行总时间
duration = time()-startTime
print("Train Finished takes:","{:.2f}".format(duration))

Train Epoch: 01 Loss= 0.153039679  Accuracy= 0.9604
Train Epoch: 02 Loss= 0.199485272  Accuracy= 0.9468
Train Epoch: 03 Loss= 0.153374299  Accuracy= 0.9632
Train Epoch: 04 Loss= 0.168705523  Accuracy= 0.9594
Train Epoch: 05 Loss= 0.145409316  Accuracy= 0.9638
Train Epoch: 06 Loss= 0.147952050  Accuracy= 0.9668
Train Epoch: 07 Loss= 0.144373626  Accuracy= 0.9712
Train Epoch: 08 Loss= 0.164091870  Accuracy= 0.9644
Train Epoch: 09 Loss= 0.172263041  Accuracy= 0.9684
Train Epoch: 10 Loss= 0.170851752  Accuracy= 0.9688
Train Epoch: 11 Loss= 0.145969361  Accuracy= 0.9702
Train Epoch: 12 Loss= 0.123259522  Accuracy= 0.9730
Train Epoch: 13 Loss= 0.154589012  Accuracy= 0.9710
Train Epoch: 14 Loss= 0.176100388  Accuracy= 0.9688
Train Epoch: 15 Loss= 0.176538453  Accuracy= 0.9702
Train Epoch: 16 Loss= 0.171091586  Accuracy= 0.9706
Train Epoch: 17 Loss= 0.184260100  Accuracy= 0.9736
Train Epoch: 18 Loss= 0.160692990  Accuracy= 0.9710
Train Epoch: 19 Loss= 0.212694019  Accuracy= 0.9716
Train Epoch:

# 评估模型

In [14]:
accu_test = sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels})
print("Test Accuracy:", accu_test)

Test Accuracy: 0.9663
